# <div align="center" style="color: #ff5733;">TSA Closure Second Notification</div>

In [1]:
# Import Libraries   

import pandas as pd  
import numpy as np  

from google.cloud import bigquery
import os

path = r"C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json"

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = path
client = bigquery.Client(project="prj-prod-dataplatform")

# Ask Balasubramani to refresh risk_credit_mis.tsa_account_closure table.
**Check if that is refreshed when Balasubramani confirms it**

In [2]:
sq = """select Run_Date, count(1)cnt from risk_credit_mis.tsa_account_closure group by 1 order by 1 desc"""
df = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')
df.head(1)

Job ID f77affd6-41d5-4f4e-88e2-20d9308a2fec successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,Run_Date,cnt
0,2024-06-18,577479


The date should show yesterday's date

# Read the data from the first sheet for Type1 and Type2 data provided by Marvin

In [3]:
d1 = pd.read_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Marvin\TSA Account Closure\April24&May24 Ongoing Batch\SecondNotification_files\Type 1  & 2  - 1st Notif.xlsx")
d1.shape

(8331, 2)

In [4]:
print(f"The unique count of customer_id is:\t{len(d1['Customer_Id'].unique())}")
d1.head()


The unique count of customer_id is:	8325


,Customer_Id,ACTION_TYPE
0,2421537,TYPE 1
1,2410303,TYPE 1
2,2435378,TYPE 1
3,2441535,TYPE 1
4,2413979,TYPE 1


# Read the data from the second sheet for Type3 data provided by Marvin

In [5]:
d2 = pd.read_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Marvin\TSA Account Closure\April24&May24 Ongoing Batch\SecondNotification_files\Type 3  - 1st Notif.xlsx")
d2.shape

(1122, 2)

In [6]:
print(f"The unique count of customer_id is:\t{len(d2['Customer_Id'].unique())}")
d2.head()

The unique count of customer_id is:	1122


,Customer_Id,ACTION_TYPE
0,2312994,TYPE 3
1,2289307,TYPE 3
2,2194639,TYPE 3
3,2192630,TYPE 3
4,2256481,TYPE 3


In [7]:
result = pd.concat([d1, d2], axis=0, ignore_index=True)
result.shape

(9453, 2)

In [8]:
result.ACTION_TYPE.value_counts()

ACTION_TYPE
TYPE 1    8304
TYPE 3    1122
TYPE 2      27
Name: count, dtype: int64

In [9]:
result[result['Customer_Id'] == '2256481']

,Customer_Id,ACTION_TYPE


In [10]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9453 entries, 0 to 9452
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Customer_Id  9453 non-null   int64 
 1   ACTION_TYPE  9453 non-null   object
dtypes: int64(1), object(1)
memory usage: 147.8+ KB


In [11]:
result['Customer_Id'] = result['Customer_Id'].astype(str)
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9453 entries, 0 to 9452
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Customer_Id  9453 non-null   object
 1   ACTION_TYPE  9453 non-null   object
dtypes: object(2)
memory usage: 147.8+ KB


In [12]:
result.to_csv("temp.csv", index = False)

In [13]:
a = tuple(result['Customer_Id'])
a[:5]

('2421537', '2410303', '2435378', '2441535', '2413979')

In [14]:
d = '2024-04-23'   ### Need to change this to D-1 everytime we run this code ###
# Get the current date
current_date = pd.Timestamp.now().date()
d_minus_1_days = current_date - pd.DateOffset(days=1)
# Calculate D-60 days
d_minus_60_days = '2024-03-27'
# Calculate D-180 days
d_minus_180_days = '2023-11-28'

print("D-1 day from current date", d_minus_1_days )
print("D-60 day from the current date - 1:", d_minus_60_days)
print("D-180 day from the current date -1:", d_minus_180_days)

d_minus_1_days_string = d_minus_1_days.strftime('%Y-%m-%d')
print("D-1 day from the current date:", d_minus_1_days_string)

d_minus_60_days_string = d_minus_60_days
print("D-60 day from the current date:", d_minus_60_days_string)

d_minus_180_days_string = d_minus_180_days
print("D-180 day from the current date:", d_minus_180_days_string)

D-1 day from current date 2024-06-18 00:00:00
D-60 day from the current date - 1: 2024-03-27
D-180 day from the current date -1: 2023-11-28
D-1 day from the current date: 2024-06-18
D-60 day from the current date: 2024-03-27
D-180 day from the current date: 2023-11-28


The date should be D-1? If the date is not D-1 then ask Bala to refresh the table.

Step1: At line 11 change the date to D-1<br><br>
Step2: At line around 36 (and (TSA_Opening_date < date '2023-10-18')   ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)) Change the date D-60 Days <br><br>
Step3: At line around 45 (WHEN tsa_status = 'Active' and Last_TSA_EOD_Balance_Date is null and (TSA_Opening_date < date '2023-10-18')  ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)) Change the date D-60 Days <br><br>
Step4: At line 58 (and TSA_Opening_date < date '2023-06-18'  and (Last_TSA_Credit_Date < date '2023-06-18' and Last_TSA_Debit_Date < date '2023-06-18' and coalesce(Last_Group_Stash_Credit_Date,date '1970-01-01') < date '2023-06-18')  ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)) <br><br>
Step5: At line 72 (Run_Date = date '2024-01-09'###DATE change here to D-1) Change the date to D-1<br><br>

First Notification we change all the steps <br><br>
Second and Third Notification we only change Step1 and Step5
For Second Notificaiton and Third Notificaiton the customre id should be from the file sent by Marvin. <br><br>

In [15]:
sq = f"""
with
w_kyc
as
(
  select distinct PARTYID, SEGCATEGORYDESC, row_number() over(partition by partyid order by SEGEFFECTIVEFROM desc) as rnk from `core_raw.party_details`
),
w_tsa
as
(
  select * from risk_credit_mis.tsa_account_closure where Run_Date = date '{d_minus_1_days_string}' ###DATE change here to D-1
  and
  Customer_Id not in (select cast(PARTYID as string) from w_kyc where rnk = 1 and SEGCATEGORYDESC = 'BKYC')
  and
  --customer_id in (select cast(customer_id as string) from prj-prod-dataplatform.dap_ds_poweruser_playground.tsa_2nd_notification_base_10jan2024)
  customer_id in {a}
)
select * from w_tsa;
"""

dfd = client.query(sq).to_dataframe()
dfd.shape

(9445, 29)

In [16]:
sq = f"""
with
w_kyc
as
(
  select distinct PARTYID, SEGCATEGORYDESC, row_number() over(partition by partyid order by SEGEFFECTIVEFROM desc) as rnk from `core_raw.party_details`
),
w_tsa
as
(
  select * from risk_credit_mis.tsa_account_closure where Run_Date = date '{d_minus_1_days_string}'  ###DATE change here to D-1
  and
  Customer_Id not in (select cast(PARTYID as string) from w_kyc where rnk = 1 and SEGCATEGORYDESC = 'BKYC')
  and
  --customer_id in (select cast(customer_id as string) from prj-prod-dataplatform.dap_ds_poweruser_playground.tsa_2nd_notification_base_10jan2024)
  customer_id in {a}
),
w_approved
as
(
  select DISTINCT customerId from risk_credit_mis.loan_master_table where loantype = 'BIGLOAN' OR FLAGAPPROVAL = 1 OR FLAGDISBURSEMENT = 1  
  or (stage1Status = 'EXPIRED' AND cddInitiatedDateAndTime IS NULL)
)
select *
from
(
  select distinct
  a.Customer_Id,a.TSA_Opening_date,
  c.OFSTANDARDACCOUNTID as TSA_Account_Number, c.OfblockAmount, 
  a.TSA_Status,a.TSA_Balance,a.Solo_Stash_Balance,a.Group_Stash_Balance,a.Term_Deposit_Balance,a.Active_Term_Deposit_flag,a.Last_TSA_Credit_Date,a.Last_TSA_Debit_Date,
  a.ARI_Flag,a.Active_Loan,a.Total_Loan_Outstanding,  case when b.customerId is not null then 1 else 0 end as Approved_Loans,
  a.Pending_Loan_Applications,a.Hard_Reject_Loans_App,a.Soft_Reject_Loans_App,a.Cancelled_Loans_App, extract(date from a.Last_Login_Date) as Last_Login_Date,
  a.Last_TSA_EOD_Balance_Date,a.Last_Group_Stash_Credit_Date,a.Group_Stash_Balance_Owner,a.Run_Date,a.Batch,a.Processing_Month,a.pending_loans_not_expired_not_exempt,a.expired_loans,
  a.pending_loans_lastDt,a.expired_loans_lastDt,
  CASE 
    WHEN tsa_status = 'Active' and (Last_TSA_Credit_Date = date '1970-01-01' and Last_Group_Stash_Credit_Date is null)
    and (TSA_Opening_date < date '{d_minus_60_days_string}')   ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
    and ( round(coalesce(TSA_Balance,0) + coalesce(Solo_Stash_Balance,0) + coalesce(Group_Stash_Balance,0) + coalesce(Term_Deposit_Balance,0),2) = 0 )
    and (Active_Loan = 0) and (COALESCE(Pending_Loan_Applications,0) = 0)
    and (Active_Term_Deposit_flag = 0)
    and (ARI_Flag = 'N')
    --and (Approved_Loans = 0)  THEN 'TYPE 1'
    and b.customerId is null  THEN 'TYPE 1'


    WHEN tsa_status = 'Active' and Last_TSA_EOD_Balance_Date is null and (TSA_Opening_date < date '{d_minus_60_days_string}')  ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
    and ( round(coalesce(TSA_Balance,0) + coalesce(Solo_Stash_Balance,0) + coalesce(Group_Stash_Balance,0) + coalesce(Term_Deposit_Balance,0),2) = 0 )
    and (Active_Loan = 0) and (COALESCE(Pending_Loan_Applications,0) = 0)
    and (Active_Term_Deposit_flag = 0)
    and (ARI_Flag = 'N')
    --and (Approved_Loans = 0)
    and b.customerId is null  
    and (date_trunc(TSA_Opening_date,day) = date_trunc(Last_Login_Date,day)) THEN 'TYPE 2'

    
    WHEN ( round(coalesce(TSA_Balance,0) + coalesce(Solo_Stash_Balance,0) + coalesce(Group_Stash_Balance,0) + coalesce(Term_Deposit_Balance,0),2) < 1 )
            and (Active_Loan = 0) and (COALESCE(Pending_Loan_Applications,0) = 0)
            and (Active_Term_Deposit_flag = 0)
            and TSA_Opening_date < date '{d_minus_180_days_string}'                                     ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
            and (Last_TSA_Credit_Date < date '{d_minus_180_days_string}'                                 ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
            and Last_TSA_Debit_Date < date '{d_minus_180_days_string}'                                         ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
            and coalesce(Last_Group_Stash_Credit_Date,date '1970-01-01') < date '{d_minus_180_days_string}' )  ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
            and tsa_status = 'Active'
            and (ARI_Flag = 'N')
    --and (Approved_Loans = 0)
            and b.customerId is null  
    THEN 'TYPE 3'
    ELSE 'NO ACTION'
    END AS ACTION_TYPE
  from w_tsa A
  left join w_approved b
  on cast(a.Customer_Id as int64) = b.customerId
  left join core_raw.customer_accounts c
  on c.OFCUSTOMERID = a.Customer_Id and c.CRINTERDESC = 'Transactional Savings Account Inv_R' and c.OFISCLOSED = 'N'
  where
  Run_Date = date '{d_minus_1_days_string}'  ###DATE change here to D-1
)
"""


In [17]:
print(sq)


with
w_kyc
as
(
  select distinct PARTYID, SEGCATEGORYDESC, row_number() over(partition by partyid order by SEGEFFECTIVEFROM desc) as rnk from `core_raw.party_details`
),
w_tsa
as
(
  select * from risk_credit_mis.tsa_account_closure where Run_Date = date '2024-06-18'  ###DATE change here to D-1
  and
  Customer_Id not in (select cast(PARTYID as string) from w_kyc where rnk = 1 and SEGCATEGORYDESC = 'BKYC')
  and
  --customer_id in (select cast(customer_id as string) from prj-prod-dataplatform.dap_ds_poweruser_playground.tsa_2nd_notification_base_10jan2024)
  customer_id in ('2421537', '2410303', '2435378', '2441535', '2413979', '2387124', '2285488', '2430530', '2433744', '2427781', '2420913', '2423475', '2418176', '2436126', '2424945', '2431247', '2436791', '2428956', '2419886', '2441219', '2414588', '2426864', '2371940', '2356900', '2436143', '2435827', '2425334', '2441462', '2414421', '2433463', '2428449', '2408787', '2423594', '2419819', '2418472', '2411332', '2411885', '2241930',

In [18]:
df = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')

Job ID 2a2c6cf7-0aaa-42a4-8d58-b6b21cc3e948 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


In [19]:
pd.set_option('display.max_columns', None)
df.sample(10)

,Customer_Id,TSA_Opening_date,TSA_Account_Number,OfblockAmount,TSA_Status,TSA_Balance,Solo_Stash_Balance,Group_Stash_Balance,Term_Deposit_Balance,Active_Term_Deposit_flag,Last_TSA_Credit_Date,Last_TSA_Debit_Date,ARI_Flag,Active_Loan,Total_Loan_Outstanding,Approved_Loans,Pending_Loan_Applications,Hard_Reject_Loans_App,Soft_Reject_Loans_App,Cancelled_Loans_App,Last_Login_Date,Last_TSA_EOD_Balance_Date,Last_Group_Stash_Credit_Date,Group_Stash_Balance_Owner,Run_Date,Batch,Processing_Month,pending_loans_not_expired_not_exempt,expired_loans,pending_loans_lastDt,expired_loans_lastDt,ACTION_TYPE
2287,2429339,2024-03-14,60824293390002,0.0,Active,0.00,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-03-14,NaT,NaT,NaN,2024-06-18,Batch_1,June,0,1,NaT,2024-06-13,TYPE 1
185,2411292,2024-02-25,60824112920007,0.0,Active,0.00,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-02-26,NaT,NaT,NaN,2024-06-18,Batch_1,June,0,1,NaT,2024-05-26,TYPE 1
6530,2440549,2024-03-25,60824405490004,0.0,Active,0.00,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,1,0,2024-03-25,NaT,NaT,NaN,2024-06-18,Batch_1,June,0,0,NaT,NaT,TYPE 1
630,2429296,2024-03-14,60824292960008,0.0,Active,0.00,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-05-04,NaT,NaT,NaN,2024-06-18,Batch_1,June,0,1,NaT,2024-06-13,TYPE 1
1702,2057860,2023-05-23,60820578600007,0.0,Active,113.53,0.0,0.0,0.0,0,2024-05-31,2023-10-31,N,0,0.0,0,1,0,0,1,2024-05-27,2024-06-18,NaT,NaN,2024-06-18,Batch_1,June,1,1,2024-05-27,2023-08-23,NO ACTION
6360,1822198,2022-12-16,60818221980006,0.0,Active,0.61,0.0,0.0,0.0,0,2023-10-31,2023-10-31,N,0,0.0,0,0,0,0,0,2024-01-02,2024-06-18,NaT,NaN,2024-06-18,Batch_1,June,0,1,NaT,2023-03-28,TYPE 3
4857,2418673,2024-03-04,60824186730004,0.0,Active,0.00,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,1,0,2024-04-17,NaT,NaT,NaN,2024-06-18,Batch_1,June,0,0,NaT,NaT,TYPE 1
172,2426479,2024-03-11,60824264790001,0.0,Active,0.00,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-03-12,NaT,NaT,NaN,2024-06-18,Batch_1,June,0,0,NaT,NaT,TYPE 1
2699,2412148,2024-02-26,60824121480004,0.0,Active,0.00,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-02-26,NaT,NaT,NaN,2024-06-18,Batch_1,June,0,1,NaT,2024-05-27,TYPE 1
6747,2411591,2024-02-25,60824115910005,0.0,Active,0.00,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-04-04,NaT,NaT,NaN,2024-06-18,Batch_1,June,0,1,NaT,2024-05-26,TYPE 1


In [20]:
df.shape

(9445, 32)

In [21]:
df['ACTION_TYPE'].value_counts()

ACTION_TYPE
TYPE 1       8173
TYPE 3        975
NO ACTION     270
TYPE 2         27
Name: count, dtype: int64

In [22]:
df.to_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Marvin\TSA Account Closure\April24&May24 Ongoing Batch\2024018_TSA_Account_Closure_2ndtNotification.xlsx", sheet_name='TSA_Closure_2ndNotification', index = False)